In [1]:
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [3]:
from langchain_core.tools import tool

In [5]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplica dos números enteros."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Suma dos números enteros."
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiza la base a la potencia del exponente."
    return base**exponent

In [6]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [8]:
from operator import itemgetter
from typing import Dict, List, Union
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

In [9]:
tools = [multiply, exponentiate, add]
tools

[StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x0000021CCA2C1BD0>),
 StructuredTool(name='exponentiate', description='exponentiate(base: int, exponent: int) -> int - Exponentiza la base a la potencia del exponente.', args_schema=<class 'pydantic.v1.main.exponentiateSchema'>, func=<function exponentiate at 0x0000021CCBFB0EE0>),
 StructuredTool(name='add', description='add(first_int: int, second_int: int) -> int - Suma dos números enteros.', args_schema=<class 'pydantic.v1.main.addSchema'>, func=<function add at 0x0000021CCA2C1E10>)]

In [10]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021CCC44ECE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021CCC4FEB00>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'first_int': {'type': 'integer'}, 'second_int': {'type': 'integer'}}, 'required': ['first_int', 'second_int']}}}, {'type': 'function', 'function': {'name': 'exponentiate', 'description': 'exponentiate(base: int, exponent: int) -> int - Exponentiza la base a la potencia del exponente.', 'parameters': {'type': 'object', 'properties': {'base': {'type': 'integer'}, 'exponent': {'type': 'integer'}}, 'required': ['base', 'exponent']}}}, {'type': 'function', 'function': {'name': 'add',

In [11]:
tool_map = {tool.name: tool for tool in tools}
tool_map

{'multiply': StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x0000021CCA2C1BD0>),
 'exponentiate': StructuredTool(name='exponentiate', description='exponentiate(base: int, exponent: int) -> int - Exponentiza la base a la potencia del exponente.', args_schema=<class 'pydantic.v1.main.exponentiateSchema'>, func=<function exponentiate at 0x0000021CCBFB0EE0>),
 'add': StructuredTool(name='add', description='add(first_int: int, second_int: int) -> int - Suma dos números enteros.', args_schema=<class 'pydantic.v1.main.addSchema'>, func=<function add at 0x0000021CCA2C1E10>)}

In [12]:
def call_tools(msg: AIMessage) -> Runnable:
    """Herramienta Simple secuencial de ayuda a la llamada."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [13]:
chain = llm_with_tools | call_tools
chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021CCC44ECE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021CCC4FEB00>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'first_int': {'type': 'integer'}, 'second_int': {'type': 'integer'}}, 'required': ['first_int', 'second_int']}}}, {'type': 'function', 'function': {'name': 'exponentiate', 'description': 'exponentiate(base: int, exponent: int) -> int - Exponentiza la base a la potencia del exponente.', 'parameters': {'type': 'object', 'properties': {'base': {'type': 'integer'}, 'exponent': {'type': 'integer'}}, 'required': ['base', 'exponent']}}}, {'type': 'function', 'function': {'name': 'add',

In [14]:
resultado = chain.invoke("¿Cuanto es 23 veces siete?")
resultado

[{'name': 'multiply',
  'args': {'first_int': 23, 'second_int': 7},
  'id': 'call_ttwa',
  'output': 161}]

In [15]:
resultado = chain.invoke("suma un millón más mil millones")
resultado

[{'name': 'add',
  'args': {'first_int': 1000000, 'second_int': 1000000000},
  'id': 'call_d3qn',
  'output': 1001000000}]

In [16]:
resultado = chain.invoke("¿Cuanto es treinta y siete al cubo?")
resultado

[{'name': 'exponentiate',
  'args': {'base': 37, 'exponent': 3},
  'id': 'call_mvk4',
  'output': 50653}]